In [379]:
import nltk
import re
import string
import math

In [380]:
## This method load the input file and split it in multiple document based on the new line
def loadFile(filename):
    file = open(filename, mode='rt', encoding='utf-8')
    text = file.read()
    file.close()
    lineList = text.splitlines()
    lineDict = {}
    for line in lineList:
        tmpLine = re.split(r'\t+', line)
        if len(tmpLine) > 1:
            lineDict[tmpLine[0]] = tmpLine[1].lower()
    return lineDict

In [381]:
## This method preprocess input file by removing punction without contraction and stemming
def preprocessAlllines(lineDict):
    
    porterStemmer = nltk.PorterStemmer()
    
    preProcessedWord = {}
    for index, value in lineDict.items():
        value = value.translate(str.maketrans('', '', string.punctuation))
        values = nltk.word_tokenize(value)
        stemmed = [porterStemmer.stem(t) for t in values]
        preProcessedWord[index] = stemmed
    return preProcessedWord

In [382]:
## This method calculates the inverted index for all document.
## It stores the inverted index in a dictionary where key is the term and value is a sorted list containing docID
def createInvertedIndex(tokenizedWordList):
    invertedIndex = {}
    documentFrequency = {}
    termFrequencyByDocument = {}
    for index, value in tokenizedWordList.items():
        for word in value:
            if word in invertedIndex.keys():
                tmpPosting =  invertedIndex[word]
                if index not in tmpPosting:
                    tmpPosting.append(index)
                    invertedIndex[word] = sorted(tmpPosting)
                    documentFrequency[word] += 1
            else:
                invertedIndex[word] = [index]
                documentFrequency[word] = 1
            if (index, word) in termFrequencyByDocument.keys():
                termFrequencyByDocument[index, word] += 1
            else:
                termFrequencyByDocument[index, word] = 1
    return (termFrequencyByDocument, {i:documentFrequency[i] for i in sorted(documentFrequency.keys())}
            , {i:invertedIndex[i] for i in sorted(invertedIndex.keys())})

In [383]:
## This method calculates intersection of two posting
def intersect(posting1, posting2):
    i = 0
    j = 0
    intersection = []
    while i != len(posting1) and j != len(posting2):
        if posting1[i] == posting2[j]:
            intersection.append(posting1[i])
            i += 1
            j += 1
        else:
            if posting1[i] < posting2[j]:
                i += 1
            else:
                j += 1
    return intersection

In [384]:
# This method merges n posting list by using intersect method if the query type is AND
# It always take the smallest posting list first based on the document frequency of each term
def intersectNPosting(termList, documentFrequency, invertedIndex):
    if len(termList) < 2:
        print("Invalid query term lsit")
        return None
    
    tmpQueryTerm = {}
    for term in termList:
        tmpQueryTerm[term] = documentFrequency[term]

    #sort to take the smallest posting list first    
    tmpQueryTerm = sorted(tmpQueryTerm.items(), key=lambda x: x[1])
    keyList = []
    for key in tmpQueryTerm:
        keyList.append(key[0]) 

    result = invertedIndex[keyList[0]]
    i = 1
    while i < len(keyList):
        result = intersect(result, invertedIndex[keyList[i]])
        i += 1
    return result       

In [385]:
## This method merges n posting list if the query type is the OR
def unionNPosting(termList, documentFrequency, invertedIndex):
    if len(termList) < 2:
        print("Invalid query term lsit")
        return None
    result = []
    i = 0
    while i < len(termList):
        if termList[i] in invertedIndex.keys():
            result = list(set(result + invertedIndex[termList[i]]))
        i += 1
    return result

In [386]:
## This method calculates the tf idf score of each query term with the retrived document list
## It returns a dictionary where key is a tuple of (docId, term) and value is the corresponding tf-idf score
def calculateTFIDFScore(termList, nDocument, termFreqByDocument, invertedIndex, retrivedResult):
    ifidfscoreList = {}
    for docId in retrivedResult:
        for term in termList:
            termIinDocI = 0
            numberOfDocWithTermI = 0
            if (docId, term) in termFreqByDocument.keys():
                termIinDocI = termFreqByDocument[docId, term]
            if term in invertedIndex.keys():
                numberOfDocWithTermI = len(invertedIndex[term])
            tf =  termIinDocI / len(nDocument[docId]) 
            idf = math.log(len(nDocument) / numberOfDocWithTermI)
            ifidfscoreList[docId, term] = tf * idf        
    return ifidfscoreList

In [387]:
## this method preprocesses the input query string and determine type of query
## it returns a list of query term 
def preProcessQuery(queryString):
    porterStemmer = nltk.PorterStemmer()
    stemmed = []
    queryType = ''
    if "AND" in queryString and "OR" in queryString:
        if "(" not in queryString:
            return queryType, stemmed
        compiler = re.compile(r'\(.*?\)')
        stemmed = compiler.findall(queryString)
        DNForCNFDeterminer = re.split('\(.*?\)', queryString)
        
        for dnf in DNForCNFDeterminer:
            if "OR" in dnf:
                queryType = "ORofAND"
                dnfs = dnf.split()
                if len(dnfs) > 1:
                    d = dnfs[0]
                    if "OR" in dnfs[1]:
                        d = dnfs[1]
                    d = d.translate(str.maketrans('', '', string.punctuation))
                    stemmed.append(porterStemmer.stem(d).lower().strip())
                    
            elif "AND" in dnf:
                queryType = "ANDofOR"
                dnfs = dnf.split()
                if len(dnfs) > 1:
                    d = dnfs[0]
                    if "AND" in dnfs[1]:
                        d = dnfs[1]
                    d = d.translate(str.maketrans('', '', string.punctuation))
                    stemmed.append(porterStemmer.stem(d).lower().strip())
                    
    elif "AND" in queryString:
        queryString = queryString.translate(str.maketrans('', '', string.punctuation))
        terms = queryString.split("AND")
        stemmed = [porterStemmer.stem(t).lower().strip() for t in terms]
        queryType = "AND"
        
    elif "OR" in queryString:
        queryString = queryString.translate(str.maketrans('', '', string.punctuation))
        terms = queryString.split("OR")
        stemmed = [porterStemmer.stem(t).lower().strip() for t in terms]
        queryType = "OR"
        
    else:
        print("Invalid query String")

    return queryType, stemmed

## Problem 1- Inverted index of document

In [388]:
preprocessAllLine = loadFile("tweets_corpus.txt")

preProcessed = preprocessAlllines(preprocessAllLine)
termFrequencyByDocument, documentFreq, invertedInd = createInvertedIndex(preProcessed)
print("Inverted Index of all document:\n", invertedInd)

Inverted Index of all document:
 {'1': ['81582996083326976'], '100': ['81844590625304576'], '1st': ['81582996083326976'], '2': ['81716618236928000'], '2nd': ['81582996083326976'], '38': ['81716618236928000'], '3rd': ['81582996083326976'], 'a': ['81499877556760576', '81534165975171072', '81535634019323904', '81582996083326976', '81587643376336896', '81600113016971264', '81623945064890368', '81644157432643584', '81673244926685184', '81715158593966080'], 'age': ['81582996083326976'], 'aint': ['81534165975171072', '81673244926685184'], 'all': ['81656304107651072'], 'also': ['81715158593966080'], 'and': ['81499877556760576', '81587643376336896', '81656304107651072', '81673244926685184', '81715158593966080', '81716618236928000', '81736742478155778'], 'ani': ['81500716438523904'], 'asparagu': ['82650970722533376'], 'ass': ['81644157432643584'], 'at': ['81644157432643584'], 'ate': ['82650970722533376'], 'bacon': ['81736742478155778', '82650970722533376', '85032815321825280', '86441828815089664

## Problem 2 - Execute Simple boolean Query
#### Supported query:
- Query in DNF form
- Query in CNF form
- Query in AND of n terms
- Query in OR of n terms (DNF)
#### Unsupported query:
- Query containing negation of term(NOT)
- nested query with nested parenthesis

In [389]:
def executeQuery(queryString):
    queryType, termList = preProcessQuery(queryString)
    result  = "No document found"
    if queryType == "AND":
        result = intersectNPosting(termList, documentFreq, invertedInd)
        
    elif queryType == "OR":
        result = unionNPosting(termList, documentFreq, invertedInd)
        
    elif queryType == "ANDofOR":
        for group in termList:
            result = []
            if len(group.strip()) > 0:
                if "AND" not in group and "OR" not in group:
                    result1 = invertedInd[group]
                else:
                    tmpqueryType, tmptermList = preProcessQuery(group)
                    result1 = unionNPosting(tmptermList, documentFreq, invertedInd)
            
                if len(result) > 0 and len(result1) > 0:
                    result = intersect(result, result1)
                elif len(result) == 0 and len(result1) > 0:
                    result = result1
                   
    elif queryType == "ORofAND":
        for group in termList:
            result = []
            if len(group.strip()) > 0:
                if "AND" not in group and "OR" not in group:
                    result1 = invertedInd[group]
                else:
                    tmpqueryType, tmptermList = preProcessQuery(group)
                    result1 = intersectNPosting(tmptermList, documentFreq, invertedInd)
                result = list(set(result + result1))
            
    print("Query:",queryString,"\nRetrived document list:\n", result, "\n")
    return result

In [390]:
result = executeQuery("(Bacon AND I) OR (Egg AND Bacon)")
result1 = executeQuery("(Bacon AND I) OR Egg")
result2 = executeQuery("Bacon OR (I AND Egg")
result3 = executeQuery("Bacon AND I AND Egg")
result4 = executeQuery("Bacon OR I OR Egg")
result5 = executeQuery("(Bacon OR I) AND (Egg OR Bacon)")
result6 = executeQuery("(Bacon OR Egg) AND I")
result7 = executeQuery("Bacon AND ( Egg OR I)")

Query: (Bacon AND I) OR (Egg AND Bacon) 
Retrived document list:
 ['81736742478155778', '86441828815089664', '82650970722533376', '85032815321825280'] 

Query: (Bacon AND I) OR Egg 
Retrived document list:
 ['81623945064890368', '81500716438523904', '85094773555335168'] 

Query: Bacon OR (I AND Egg 
Retrived document list:
 ['81623945064890368', '81500716438523904', '85094773555335168'] 

Query: Bacon AND I AND Egg 
Retrived document list:
 ['82650970722533376', '86441828815089664'] 

Query: Bacon OR I OR Egg 
Retrived document list:
 ['81716618236928000', '82650970722533376', '81842384404623360', '81600113016971264', '81736742478155778', '81587643376336896', '81673244926685184', '81644157432643584', '81500716438523904', '86441828815089664', '85032815321825280', '81503002321616896'] 

Query: (Bacon OR I) AND (Egg OR Bacon) 
Retrived document list:
 ['82650970722533376', '81736742478155778', '81587643376336896', '81673244926685184', '86441828815089664', '85032815321825280', '81503002321

## Supported and Unsupported query type
The executeQuery() supports following types of query:
#### Supported query:
- Query in DNF form
- Query in CNF form
- Query in AND of n terms
- Query in OR of n terms (DNF)
#### Unsupported query:
- Query containing negation of term(NOT)
- nested query with nested parenthesis

## Problem 3- TF-IDF score of retrived document from problem 2

In [391]:
tfIdfScoreOfRetrivedDocument = calculateTFIDFScore(termList, preProcessed
                                                   , termFrequencyByDocument, invertedInd, result)
tfIdfScoreOfRetrivedDocument1 = calculateTFIDFScore(termList, preProcessed
                                                   , termFrequencyByDocument, invertedInd, result1)
tfIdfScoreOfRetrivedDocument2 = calculateTFIDFScore(termList, preProcessed
                                                   , termFrequencyByDocument, invertedInd, result2)
tfIdfScoreOfRetrivedDocument3 = calculateTFIDFScore(termList, preProcessed
                                                   , termFrequencyByDocument, invertedInd, result3)
print("TFIDF Score:\n", tfIdfScoreOfRetrivedDocument, "\n")
print("TFIDF Score:\n", tfIdfScoreOfRetrivedDocument1, "\n")
print("TFIDF Score:\n", tfIdfScoreOfRetrivedDocument2, "\n")
print("TFIDF Score:\n", tfIdfScoreOfRetrivedDocument3, "\n")
tfIdfScoreOfRetrivedDocument4 = calculateTFIDFScore(termList, preProcessed
                                                   , termFrequencyByDocument, invertedInd, result4)
tfIdfScoreOfRetrivedDocument5 = calculateTFIDFScore(termList, preProcessed
                                                   , termFrequencyByDocument, invertedInd, result5)
tfIdfScoreOfRetrivedDocument6 = calculateTFIDFScore(termList, preProcessed
                                                   , termFrequencyByDocument, invertedInd, result6)
tfIdfScoreOfRetrivedDocument7 = calculateTFIDFScore(termList, preProcessed
                                                   , termFrequencyByDocument, invertedInd, result7)
print("TFIDF Score:\n", tfIdfScoreOfRetrivedDocument4, "\n")
print("TFIDF Score:\n", tfIdfScoreOfRetrivedDocument5, "\n")
print("TFIDF Score:\n", tfIdfScoreOfRetrivedDocument6, "\n")
print("TFIDF Score:\n", tfIdfScoreOfRetrivedDocument7, "\n")

TFIDF Score:
 {('81736742478155778', 'bacon'): 0.1791759469228055, ('81736742478155778', 'i'): 0.0, ('81736742478155778', 'egg'): 0.12321436812926323, ('86441828815089664', 'bacon'): 0.08532187948705024, ('86441828815089664', 'i'): 0.04670615490532029, ('86441828815089664', 'egg'): 0.058673508632982485, ('82650970722533376', 'bacon'): 0.08532187948705024, ('82650970722533376', 'i'): 0.04670615490532029, ('82650970722533376', 'egg'): 0.058673508632982485, ('85032815321825280', 'bacon'): 0.11945063128187033, ('85032815321825280', 'i'): 0.0, ('85032815321825280', 'egg'): 0.08214291208617548} 

TFIDF Score:
 {('81623945064890368', 'bacon'): 0.0, ('81623945064890368', 'i'): 0.0, ('81623945064890368', 'egg'): 0.0, ('81500716438523904', 'bacon'): 0.0, ('81500716438523904', 'i'): 0.14011846471596087, ('81500716438523904', 'egg'): 0.0, ('85094773555335168', 'bacon'): 0.0, ('85094773555335168', 'i'): 0.0, ('85094773555335168', 'egg'): 0.0} 

TFIDF Score:
 {('81623945064890368', 'bacon'): 0.0, ('